In [113]:
import pandas as pd

df = pd.read_csv('issuer-mfm-mappings.csv')

In [116]:
stop_words_normalized = [
        "healthcare",
        "technologies",
        "therapeutics",
        "financial",
        "software",
        "holdings",
        "transportation",
        "pharmaceuticals",
        "capital",
        "copper",
        "communications",
        "biotechnology",
        "biopharmaceuticals",
        "group",
        "technology",
        "media",
        "energy",
        "industries",
        "biotherapeutics",
        "solution",
        "bioscience",
        "industries",
        "corporation",
        "systems",
        "enterprises",
        "robotics",
        "bank",
        "inc",
        "llc",
        "pp",
        "series a",
        "series seed",
        "series b",
        "series c",
        "series d",
        "series e",
        "series f",
        "series g",
        "series h",
        "series i"
    ]

def normalize_name(name):
    name = name.lower() 
    # print(name)
    for word in stop_words_normalized:
        name = name.replace(word, '')
        # print(word, name)
    return name.strip()

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

def remove_stop_words(sentence):
    words = sentence.split()
    filtered_sentence = [w for w in words if not w.lower() in stop_words]
    return ' '.join(filtered_sentence)


test_word_to_find = df['nport_security_name'][0]
word_list = df['slug']
print(test_word_to_find)

test_word_to_find_normalzed = normalize_name(test_word_to_find)
print(test_word_to_find_normalzed)

word_to_find_no_stop_word = remove_stop_words(test_word_to_find_normalzed)
print(word_to_find_no_stop_word)

Arcellx, Inc.,,
arcellx, .,,
arcellx, .,,


In [55]:
import string
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('', '', string.punctuation))

word_to_find_no_punc = remove_punctuation(word_to_find_no_stop_word)
print(word_to_find_no_punc)

arcellx 


In [56]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

def stem_words(word):
    return stemmer.stem(word)

word_to_find_stemmed = stem_words(word_to_find_no_punc)
print(word_to_find_stemmed)

arcellx 


In [108]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(word_to_find):
    

    word_to_find_normalzed = normalize_name(word_to_find)
    # print(word_to_find_normalzed)

    word_to_find_no_stop_word = remove_stop_words(word_to_find_normalzed)
    # print(word_to_find_no_stop_word)

    word_to_find_no_punc = remove_punctuation(word_to_find_no_stop_word)
    # print(word_to_find_no_punc)
   
    # word_to_find_stemmed = stem_words(word_to_find_no_punc)
    # print(word_to_find_stemmed)
 
 
    return word_to_find_no_punc
  


In [117]:
def find_partial_words_in_list(phrase, word_list):
    cleaned_words = clean_text(phrase).split()
    matched_words = []

    for word in cleaned_words:
        # Escape special regex characters
        regex_pattern = re.escape(word)
        compiled_pattern = re.compile(regex_pattern, re.IGNORECASE)
        
        # Find partial matches in the word list
        partial_matches = [w for w in word_list if compiled_pattern.search(clean_text(w))]
        
        # Add the partial matches to the final result list
        matched_words.extend(partial_matches)
    
    # Return unique matched words
    return list(set(matched_words))

test_word_to_find = df['nport_security_name'][0]
word_list = df['slug']

matches = find_partial_words_in_list(test_word_to_find, word_list)

print(test_word_to_find)

print("Matched words:", matches)


Arcellx, Inc.,,
Matched words: ['arcellx']


In [118]:
def find_matches(phrase, word_list, exact_match_weight=3):
    cleaned_words = clean_text(phrase).split()
    
    # A dictionary to keep track of match scores
    match_scores = {w: 0 for w in word_list}
    
    for word in cleaned_words:
        regex_pattern = re.escape(word)
        print(regex_pattern)
        compiled_pattern = re.compile(regex_pattern, re.IGNORECASE)
        print(compiled_pattern)
        
        # Iterate through the word list and conut matches
        for list_item in word_list:
            if compiled_pattern.search(clean_text(list_item)):
                match_scores[list_item] += 1

    # Add exact match weight if the whole phrase is found
    for list_item in word_list:
        if re.search(re.escape(clean_text(phrase)), clean_text(list_item)):
            match_scores[list_item] += exact_match_weight
    
    # Rank the results based on the number of matches
    ranked_results = sorted(match_scores.items(), key=lambda x: x[1], reverse=True)
    
    return ranked_results

test_word_to_find = df['nport_security_name'][0]
word_list = df['slug']
matches = find_matches(test_word_to_find, word_list, exact_match_weight=3)

ranked_matches = [match for match in matches if match[1] > 0]

print("Ranked matches:", ranked_matches)


arcellx
re.compile('arcellx', re.IGNORECASE)
Ranked matches: [('arcellx', 5)]


In [119]:
# match with weights on first word

def find_matches(phrase, word_list, exact_match_weight=3, first_word_weight=2, other_word_weight=1):
    cleaned_words = clean_text(phrase).split()
    match_scores = {w: 0 for w in word_list}
    
    # Handle the first word with a higher weight
    if cleaned_words:
        first_word = cleaned_words[0]
        first_word_regex = re.escape(first_word)
        print(first_word_regex)
        compiled_first_word_pattern = re.compile(first_word_regex, re.IGNORECASE)
        print(compiled_first_word_pattern)
        
        # Add higher weight for first word matches
        for list_item in word_list:
            if compiled_first_word_pattern.search(clean_text(list_item)):
                match_scores[list_item] += first_word_weight
    
    # Handle other words with a standard weight
    for word in cleaned_words[1:]:  
        regex_pattern = re.escape(word)
        print(regex_pattern)
        compiled_pattern = re.compile(regex_pattern, re.IGNORECASE)       
        print(compiled_pattern)
        
        for list_item in word_list:
            if compiled_pattern.search(clean_text(list_item)):
                match_scores[list_item] += other_word_weight

    # Add exact match weight if the whole phrase is found
    for list_item in word_list:
        if re.search(re.escape(clean_text(phrase)), clean_text(list_item)):
            match_scores[list_item] += exact_match_weight
    
    # Rank the results based on the number of matches
    ranked_results = sorted(match_scores.items(), key=lambda x: x[1], reverse=True)
    
    return ranked_results


test_word_to_find = df['nport_security_name'][0]
matches = find_matches(test_word_to_find, word_list, exact_match_weight=3, first_word_weight=2, other_word_weight=1)

# Filter only the results that have at least 1 match
ranked_matches = [match for match in matches if match[1] > 0]

print("Ranked matches:", ranked_matches)


arcellx
re.compile('arcellx', re.IGNORECASE)
Ranked matches: [('arcellx', 10)]


In [132]:
words_to_find = df.apply(lambda row: row['nport_issuer_name'] if pd.notna(row['nport_issuer_name']) else row['nport_security_name'], axis=1)
# print(words_to_find)
word_list = df['slug']

In [123]:
from fuzzywuzzy import fuzz, process
def match_company(name, master_list):
    name = normalize_name(name)
    best_match = process.extractOne(name, master_list, scorer=fuzz.token_set_ratio)
    return best_match[0] if best_match else None

In [127]:

matched_companies = [match_company(c, word_list) for c in words_to_find]



In [128]:
matches = sum(1 for a, b in zip(matched_companies, word_list) if a == b)
    
accuracy = matches / len(word_list)

print(len(word_list))
print(f"Number of matches: {matches}")
print(f"Accuracy: {accuracy:.2%}")

3108
Number of matches: 2886
Accuracy: 92.86%


In [ ]:
# ranked fuzzy
from fuzzywuzzy import fuzz, process

def match_company(name, master_list, top_n=5):
    name = normalize_name(name)
    
    # Extract the top N matches along with their scores
    ranked_matches = process.extract(name, master_list, scorer=fuzz.token_set_ratio, limit=top_n)
    return ranked_matches if ranked_matches else []

matched_companies = [match_company(c, word_list) for c in words_to_find]

In [111]:
exact_matches = 0
partial_matches = 0
total_items = len(word_list)


for ranked_match_list, expected in zip(matched_companies, word_list):
    if ranked_match_list:
        first_match = ranked_match_list[0][0]  # Get the first-ranked match
        
        # Exact match
        if first_match == expected:
            exact_matches += 1
        # Partial match (i.e., the expected company appears anywhere else in the ranked list)
        elif any(match[0] == expected for match in ranked_match_list):
            partial_matches += 1


exact_match_accuracy = exact_matches / total_items if total_items > 0 else 0
overall_match_accuracy = (exact_matches + partial_matches) / total_items if total_items > 0 else 0


print(f"Total items: {total_items}")
print(f"Number of exact matches: {exact_matches}")
print(f"Number of partial matches: {partial_matches}")
print(f"Exact Match Accuracy: {exact_match_accuracy:.2%}")
print(f"Overall Accuracy (including partial matches): {overall_match_accuracy:.2%}")


Total items: 3108
Number of exact matches: 2886
Number of partial matches: 46
Exact Match Accuracy: 92.86%
Overall Accuracy (including partial matches): 94.34%


In [129]:
non_matches = [(a, b, c) for a, b, c in zip(matched_companies, word_list, words_to_find) if a != b]
print("Non-matching items:")
for matched, original, words_to_find in non_matches:
    print(f"Matched: {matched}, SLUG: {original}, Words_to_find: {words_to_find}")

Non-matching items:
Matched: clear-street, SLUG: byju-s, Words_to_find: Think & Learn Private Ltd.
Matched: honor, SLUG: doordash, Words_to_find: DOOR DASH
Matched: neurogene, SLUG: neogene-therapeutics, Words_to_find: NEOGEN CORPORATION
Matched: rover, SLUG: rapport-therapeutics, Words_to_find: RAPPORT THERAPEUTICS INC
Matched: xad, SLUG: warby-parker, Words_to_find: JAND INC
Matched: trisalus-life-sciences, SLUG: 1life-healthcare, Words_to_find: Life Healthcare Group Holdings Ltd
Matched: fanatics, SLUG: 3d-robotics, Words_to_find: Nauticus Robotics, Inc.
Matched: nextroll, SLUG: adroll, Words_to_find: NEXTROLL, INC. COMMON SHARES   /
Matched: nextroll, SLUG: adroll, Words_to_find: NEXTROLL COMMON SHARES   /
Matched: nextroll, SLUG: adroll, Words_to_find: Nextroll, Inc. Shares /
Matched: 23andme, SLUG: advantagene, Words_to_find: Candel Therapeutics Inc.
Matched: 23andme, SLUG: advantagene, Words_to_find: CANDEL THERAPEUTICS INC
Matched: coursera, SLUG: airtable, Words_to_find: Forma

In [130]:
print(non_matches[1])

('honor', 'doordash', 'DOOR DASH')


In [101]:
import pandas as pd

not_matched = [t for t in non_matches]  

df_rows = []

for t in not_matched:
    not_matched_to_find = t[2]  # Use nport_issuer_name for find_matches
    matches = find_matches(not_matched_to_find, word_list, exact_match_weight=3)

    ranked_matches = [match for match in matches if match[1] > 0]

    # Convert ranked matches to string for storage in df
    ranked_matches_str = ', '.join([f"('{item[0]}', {item[1]})" for item in ranked_matches])

    # Append the slug, nport_issuer_name and ranked_matches to df_rows
    df_rows.append([t[1], t[2], ranked_matches_str])

df = pd.DataFrame(df_rows, columns=['slug', 'nport_issuer_name', 'Ranked Matches'])

df.to_csv('data/output.csv', index=False)

print("Data saved to 'data/output.csv'.")


think
re.compile('think', re.IGNORECASE)
learn
re.compile('learn', re.IGNORECASE)
private
re.compile('private', re.IGNORECASE)
ltd
re.compile('ltd', re.IGNORECASE)
door
re.compile('door', re.IGNORECASE)
dash
re.compile('dash', re.IGNORECASE)
neogen
re.compile('neogen', re.IGNORECASE)
raort
re.compile('raort', re.IGNORECASE)
jand
re.compile('jand', re.IGNORECASE)
life
re.compile('life', re.IGNORECASE)
ltd
re.compile('ltd', re.IGNORECASE)
nauticus
re.compile('nauticus', re.IGNORECASE)
nextroll
re.compile('nextroll', re.IGNORECASE)
common
re.compile('common', re.IGNORECASE)
shares
re.compile('shares', re.IGNORECASE)
nextroll
re.compile('nextroll', re.IGNORECASE)
common
re.compile('common', re.IGNORECASE)
shares
re.compile('shares', re.IGNORECASE)
nextroll
re.compile('nextroll', re.IGNORECASE)
shares
re.compile('shares', re.IGNORECASE)
candel
re.compile('candel', re.IGNORECASE)
candel
re.compile('candel', re.IGNORECASE)
formagrid
re.compile('formagrid', re.IGNORECASE)
ser
re.compile('ser',

In [103]:
# do the matching on the ones that are not matched
not_matched = [t for t in non_matches]  


correct_first_match = 0
partial_matches = 0
total_items = len(not_matched)

unmatched_items = []
df_rows = []

for t in not_matched:
    second_item = t[1]  # slug
    not_matched_to_find = t[2]  # nport_issuer_name
    
    # Get ranked matches
    matches = find_matches(not_matched_to_find, word_list, exact_match_weight=3)
    
    ranked_matches = [match for match in matches if match[1] > 0]
    ranked_matches_str = ', '.join([f"('{item[0]}', {item[1]})" for item in ranked_matches])
    
    match_status = ""

    # Check the first match
    if ranked_matches:
        first_match = ranked_matches[0][0]
        
        if first_match == second_item:
            # Exact match in the first position
            correct_first_match += 1
            match_status = "Matched"
        else:
            # If second itme exists in the rest of the ranked matches
            if any(match[0] == second_item for match in ranked_matches[1:]):
                partial_matches += 1
                match_status = "Partial Match"
            else:
                unmatched_items.append(t)
                match_status = "Not Matched"
    else:
        unmatched_items.append(t)
        match_status = "Not Matched"

    df_rows.append([second_item, not_matched_to_find, ranked_matches_str, match_status])

df = pd.DataFrame(df_rows, columns=['slug', 'nport_issuer_name', 'Ranked Matches', 'Match Status'])

df.to_csv('data/output.csv', index=False)

first_match_accuracy = correct_first_match / total_items if total_items > 0 else 0
overall_match_accuracy = (correct_first_match + partial_matches) / total_items if total_items > 0 else 0

print(f"First Match Accuracy: {first_match_accuracy:.2f}")
print(f"Overall Match Accuracy (including partial matches): {overall_match_accuracy:.2f}")
print("Unmatched items:", unmatched_items)


think
re.compile('think', re.IGNORECASE)
learn
re.compile('learn', re.IGNORECASE)
private
re.compile('private', re.IGNORECASE)
ltd
re.compile('ltd', re.IGNORECASE)
door
re.compile('door', re.IGNORECASE)
dash
re.compile('dash', re.IGNORECASE)
neogen
re.compile('neogen', re.IGNORECASE)
raort
re.compile('raort', re.IGNORECASE)
jand
re.compile('jand', re.IGNORECASE)
life
re.compile('life', re.IGNORECASE)
ltd
re.compile('ltd', re.IGNORECASE)
nauticus
re.compile('nauticus', re.IGNORECASE)
nextroll
re.compile('nextroll', re.IGNORECASE)
common
re.compile('common', re.IGNORECASE)
shares
re.compile('shares', re.IGNORECASE)
nextroll
re.compile('nextroll', re.IGNORECASE)
common
re.compile('common', re.IGNORECASE)
shares
re.compile('shares', re.IGNORECASE)
nextroll
re.compile('nextroll', re.IGNORECASE)
shares
re.compile('shares', re.IGNORECASE)
candel
re.compile('candel', re.IGNORECASE)
candel
re.compile('candel', re.IGNORECASE)
formagrid
re.compile('formagrid', re.IGNORECASE)
ser
re.compile('ser',

In [133]:
# What if we clean before fuzzy match
cleaned_words_to_find = [clean_text(item) for item in words_to_find]
cleaned_master_list = [clean_text(item) for item in word_list]

matched_companies = [match_company(c, cleaned_master_list) for c in cleaned_words_to_find]
matches = sum(1 for a, b in zip(matched_companies, cleaned_master_list) if a == b)
    
accuracy = matches / len(cleaned_master_list)

print(len(cleaned_master_list))
print(f"Number of matches: {matches}")
print(f"Accuracy: {accuracy:.2%}")

3108
Number of matches: 2783
Accuracy: 89.54%
